In [146]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv(r'C:\Users\omar\Downloads\Kaggle-Digit-Recognizer-master\Kaggle-Digit-Recognizer-master\train.csv')

In [147]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape
X_train.shape

(784, 41000)

2 layers

In [137]:
def init_params():
    W1 = np.random.uniform(-0.5, 0.5, (10, 784))
    W2 = np.random.uniform(-0.5, 0.5, (10, 10))
    b1 = np.random.uniform(-0.5, 0.5, (10, 1))
    b2 = np.random.uniform(-0.5, 0.5, (10, 1))
    return W1, b1, W2, b2

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def sigmoid_deriv(Z):
    return (1/(1+np.exp(-Z)))*(1-(1/(1+np.exp(-Z))))

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = sigmoid(Z1)
    
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)

    return Z1, A1, Z2, A2



def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * sigmoid_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2 
       
    return W1, b1, W2, b2


def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent_2layers(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2= update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
    predictions = get_predictions(A2)
    print(get_accuracy(predictions, Y_train)*100)
    return W1, b1, W2, b2

In [138]:
W1, b1, W2, b2 = gradient_descent_2layers(X_train, Y_train, 0.10, 500)

[1 3 7 ... 8 8 0] [1 8 7 ... 8 8 0]
71.19268292682926


3 layers

In [165]:
def init_params():
    W1 = np.random.uniform(-0.5, 0.5, (10, 784))
    W2 = np.random.uniform(-0.5, 0.5, (10, 10))
    W3 = np.random.uniform(-0.5, 0.5, (10, 10))
    b1 = np.random.uniform(-0.5, 0.5, (10, 1))
    b2 = np.random.uniform(-0.5, 0.5, (10, 1))
    b3 = np.random.uniform(-0.5, 0.5, (10, 1))
    return W1, b1, W2, b2, W3, b3

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def sigmoid_deriv(Z):
    return (1/(1+np.exp(-Z)))*(1-(1/(1+np.exp(-Z))))

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = sigmoid(Z1)
    
    Z2 = W2.dot(A1) + b2
    A2 = sigmoid(Z2)

    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3



def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    
    # Backpropagation for the third layer
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)
    
    # Backpropagation for the second layer
    dZ2 = W3.T.dot(dZ3) * sigmoid_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    
    # Backpropagation for the first layer
    dZ1 = W2.T.dot(dZ2) * sigmoid_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    
    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2 
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3    
    return W1, b1, W2, b2, W3, b3


def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent_3layers(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        # if i % 10 == 0:
        #     continue
        #     # print("Iteration: ", i)
    predictions = get_predictions(A3)
    print(get_accuracy(predictions, Y_train)*100)
    return W1, b1, W2, b2, W3, b3

In [166]:
W1, b1, W2, b2, W3, b3 = gradient_descent_3layers(X_train, Y_train, 0.10, 3000)

[4 8 2 ... 4 8 9] [4 8 5 ... 4 3 4]
78.53902439024391


In [160]:
# 71.47560975609755